In [12]:
# Usando dados como pontos marcados, sofridos, eficiência ofensiva/defensiva e rebotes, prever quantas vitórias uma equipa terá numa temporada

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np


# 1️ Ler dados originais (sem tocar neles)
teams_df = pd.read_csv("data/teams.csv")

# 2️ Criar uma cópia para manipulação
teams_model = teams_df.copy()

# 3️ Criar features derivadas úteis para prever vitórias
teams_model["point_diff"] = teams_model["o_pts"] - teams_model["d_pts"]
teams_model["off_efficiency"] = teams_model["o_pts"] / teams_model["o_fga"]
teams_model["def_efficiency"] = teams_model["d_pts"] / teams_model["d_fga"]
teams_model["rebalance"] = teams_model["o_reb"] - teams_model["d_reb"]

#Remove linhas com valores nulos em colunas importantes.
teams_model = teams_model.dropna(subset=["won", "o_pts", "d_pts"])


teams_model.to_csv("data_models/teams_model_geral.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np

# 1️⃣ Ler os dados das jogadoras por equipa e ano
players_teams = pd.read_csv("data/players_teams.csv")

# 2️⃣ Criar uma métrica de rating individual (como já fizeste)
players_teams = (
    players_teams.groupby(["playerID", "year"], as_index=False)
    .agg({
        "points": "sum",
        "rebounds": "sum",
        "assists": "sum",
        "steals": "sum",
        "blocks": "sum",
        "turnovers": "sum",
        "tmID": "last",  # equipa final
        "minutes": "sum"
    })
)
players_teams["player_rating"] = (
    players_teams["points"] * 1 + 
    players_teams["rebounds"] * 1.2 + 
    players_teams["assists"] * 1.5 + 
    players_teams["steals"] * 3 +    
    players_teams["blocks"] * 3 -
    players_teams["turnovers"] * 2
)   / players_teams["minutes"]  # normalizar por minutos jogados

# 3️⃣ Selecionar apenas colunas relevantes
players_teams = players_teams[["playerID", "year", "tmID", "player_rating"]]

# 4️⃣ Criar coluna com o rating do ano anterior de cada jogadora
players_teams = players_teams.sort_values(["playerID", "year"])
players_teams["player_rating_prev"] = players_teams.groupby("playerID")["player_rating"].shift(1)

# 6️⃣ Agregar a nível de equipa e ano — múltiplas estatísticas
team_prev_ratings = (
    players_teams.groupby(["year", "tmID"], as_index=False)
    .agg(
        team_avg_player_prev_rating=("player_rating_prev", "mean"), #Media do rating dos jogadores da equipa no ano anterior -> qualidade geral da equipa
        team_median_player_prev_rating=("player_rating_prev", "median"),#Mediana do rating dos jogadores da equipa no ano anterior -> combater estrelas na equipa
        team_std_player_prev_rating=("player_rating_prev", "std"),  #Desvio padrão do rating dos jogadores da equipa no ano anterior -> consistência da equipa
        team_num_players_prev=("player_rating_prev", "count") #Número de jogadores com rating no ano anterior -> jogadoras experientes na equipa -> anti novatas
    )
)

# 6️⃣ Remover anos iniciais (sem dados anteriores)
#team_prev_ratings = team_prev_ratings.dropna(subset=["team_sum_player_prev_rating"])

# 7️⃣ Guardar para análise posterior
import os
os.makedirs("data_models", exist_ok=True)
team_prev_ratings.to_csv("data_models/team_prev_ratings.csv", index=False)
print(players_teams.query("year == 1 and tmID in ['SEA', 'UTA']")[["playerID","year","tmID","player_rating","player_rating_prev"]].head(10))


print("\n✅ Criada tabela com força da equipa (baseada no ano anterior):")
print(team_prev_ratings.head(10))


       playerID  year tmID  player_rating  player_rating_prev
53   aycocan01w     1  SEA       0.923077                 NaN
55    azzije01w     1  UTA       0.583542                 NaN
81   barnequ01w     1  SEA       0.555035                 NaN
230  campbed01w     1  SEA       0.602549                 NaN
408  dydekma01w     1  UTA       0.978710                 NaN
417  edwarmi01w     1  SEA       0.459110                 NaN
419  edwarsi01w     1  SEA       0.566512                 NaN
502  fresest01w     1  UTA       0.419369                 NaN
515  gaithka01w     1  UTA       0.724359                 NaN
522  garnean01w     1  SEA       0.501964                 NaN

✅ Criada tabela com força da equipa (baseada no ano anterior):
   year tmID  team_avg_player_prev_rating  team_median_player_prev_rating  \
0     1  CHA                          NaN                             NaN   
1     1  CLE                          NaN                             NaN   
2     1  DET           